# Asertos

En esencia, la sentencia assert de Python es una ayuda para la depuración que comprueba una condición: 

* Si la condición assert es verdadera, no pasa nada, y su programa continúa ejecutándose normalmente. 

* Si la condición se evalúa como falsa, se lanza una excepción AssertionError con un mensaje de error opcional.

He aquí un ejemplo sencillo para que puedas ver dónde pueden ser útiles los asertos. 

Supongamos que estás construyendo una tienda online con Python. Estás trabajando para añadir una funcionalidad de cupón de descuento al sistema, y eventualmente escribes la siguiente función apply_discount:

In [1]:
def aplicarDescuento(producto, descuento):
    precio = int(producto['precio'] * (1.0 - descuento)) 
    assert 0 <= precio <= producto['precio']
    return precio

La aserción garantizará que, pase lo que pase, los precios descontados calculados por esta función no pueden ser inferiores a 0 ni superiores al precio del producto.

In [2]:
zapatos = {'nombre': 'zapatos', 'precio': 14900}

In [3]:
aplicarDescuento(zapatos, 0.25)

11175

In [4]:
aplicarDescuento(zapatos, 2.0)

AssertionError: 

**El uso adecuado de los asertos es informar a los desarrolladores sobre errores irrecuperables en un programa.** 

Los asertos no están pensadas para señalar condiciones de error esperadas, como un error File-Not-Found, donde un usuario puede tomar acciones correctivas o simplemente intentarlo de nuevo.

**Los asertos están pensados para ser autocomprobaciones internas de tu programa. Funcionan declarando algunas condiciones como imposibles en tu código. Si una de estas condiciones no se cumple, significa que hay un error en el programa.**

Si tu programa está libre de errores, estas condiciones nunca ocurrirán. Pero si ocurren, el programa se bloqueará con un error de aserción que te dirá exactamente qué condición "imposible" se activó. Esto hace que sea mucho más fácil rastrear y arreglar los errores en tus programas. 

**Por ahora, ten en cuenta que la sentencia assert de Python es una ayuda para la depuración, no un mecanismo para manejar errores en tiempo de ejecución.**

### No hay que usar asertos para validar los datos

La mayor advertencia sobre el uso de asertos en Python es que los asertos se pueden desactivar globalmente con los modificadores de línea de comandos -O y -OO, así como con la variable de entorno PYTHONOPTIMIZE en CPython.

Esta es una decisión de diseño intencionada que se utiliza de forma similar en muchos otros lenguajes de programación. Como efecto secundario, resulta muy útil utilizar las sentencias assert como una forma rápida y sencilla de validar los datos de entrada.

In [5]:
def borrarProducto(prod_id, usuario):
    assert usuario.is_admin(), 'Debe ser administrador'
    assert tienda.tieneProducto(prod_id), 'Producto Desconocido' 
    tienda.getProducto(prod_id).delete()

**Comprobar los privilegios de administrador con un aserto es peligroso.** 

Si los asertos están deshabilitados en el intérprete de Python, esto se convierte en una operación nula. Por lo tanto, cualquier usuario puede ahora eliminar productos. La comprobación de privilegios ni siquiera se ejecuta. Esto probablemente introduce un problema de seguridad y abre la puerta a que los atacantes destruyan o dañen gravemente los datos de nuestra tienda online. No es bueno.

**La comprobación de tieneProducto() se salta cuando los asertos están deshabilitados.**

Esto significa que tieneProducto() puede ser llamado con IDs de productos inválidos, lo que podría llevar a errores más severos, dependiendo de cómo esté escrito nuestro programa. 

En el peor de los casos, esto podría ser una vía para que alguien lance ataques de denegación de servicio contra nuestra tienda. Por ejemplo, si la aplicación de la tienda se bloquea si alguien intenta eliminar un producto desconocido, un atacante podría bombardearla con solicitudes de eliminación no válidas y provocar una interrupción.

¿Cómo podemos evitar estos problemas? La respuesta es no utilizar nunca asertos para hacer la validación de datos. En su lugar, podríamos hacer nuestra validación con sentencias if regulares y lanzar excepciones de validación si es necesario, así:

In [6]:
def borrarProducto(product_id, usuario): 
    if not usuario.is_admin():
        raise AuthError('Debe ser administrador para eliminar un producto') 
    if not tienda.has_product(product_id):
        raise ValueError('Id del producto desconocida') 
    store.getProducto(product_id).delete()

### Asertos que nunca fallan

Es sorprendentemente fácil escribir accidentalmente sentencias assert de Python que siempre se evalúan como verdaderas.

Este es el problema, en pocas palabras:
    
**Cuando pasas una tupla como primer argumento en una sentencia assert, la aserción siempre se evalúa como verdadera y por lo tanto nunca falla.**

In [7]:
assert(1 == 2, 'Esto debería fallar')

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-7-e707a81cdd24>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(1 == 2, 'Esto debería fallar')


Esto tiene que ver con que las tuplas no vacías siempre son verdaderas en Python. 

**Si pasas una tupla a una sentencia assert, la condición assert siempre es verdadera, lo que a su vez hace que la sentencia assert sea inútil porque nunca puede fallar y provocar una excepción.**

Es relativamente fácil escribir accidentalmente asertos de varias líneas debido a este comportamiento poco intuitivo. 

Imagina que tienes esta afirmación en una de tus pruebas unitarias:

In [8]:
contador = 10

assert(contador == 10, 'Debería haber contado todos los elementos')

<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-8-2d6914216657>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(contador == 10, 'Debería haber contado todos los elementos')


A primera vista, este caso de prueba parece completamente correcto. 

Sin embargo, nunca detectaría un resultado incorrecto: 

**La afirmación siempre se evalúa como True, independientemente del estado de la variable del contador.**

¿Y por qué? 

**Porque afirma el valor de verdad de un objeto tupla.**

## Claves

* La sentencia assert de Python es una ayuda para la depuración que comprueba una condición como una autocomprobación interna en su programa.
* Los asertos sólo deben usarse para ayudar a los desarrolladores a identificar errores. No son un mecanismo para manejar errores en tiempo de ejecución.
* Los asertos pueden ser globalmente desactivadas con una configuración del intérprete.